## Essay description

Your friend Amy is really excited that they are learning Python and Object Orientation! Amyhey just developed a simple calculator using Python, following everything they learned: classes, and methods, and all this cool stuff. Amy asked for your help to test it. With your experience, the first thing you asked was whether your friend had unit tests for the calculator; and the answer was "No".

So, you decided to help Amy by creating a set of test cases that would help find potential bugs in the code. You looked at the code, and you knew that the code has a few issues that can be caught by unit tests (despite the questionable design decisions).

In this assignment, you are asked to:
- Write, at least, 3 test cases per function, explaining what these test cases do. 
- Include tests to cover the known cases, edge cases, and possible exceptions. 
- Run the tests and re-write the class to fix the errors you find. Create a comment in the code to say what was the issue, how you caught it, and how you fixed it.
- Explore, at least, 6 different types of asserts. It is mandatory to test exceptions (assertRaises), at least twice.
- You don't have to create tests for the functions prefixed with `print_` if you don't want to.
- You have to submit (as part of the notebook): the unit tests, and the code with the fixes and comments.

Your friend's code is presented in the two following cells:


In [1]:
import os

class Calculator:

    num1 = 0.0
    num2 = 0.0
    operation =  '+'
    result = 0.0
    
    def __init__(self, num1, num2):
        self.num1 = num1
        self.num2 = num2

    def sum(self): 
        self.operation = '+'
        self.result = self.num1 + self.num2
        return self.result


    def subtract(self): 
        self.operation = '-'
        self.result = self.num1 - self.num2
        return self.result


    def multiply(self): 
        self.operation = '*'
        self.result = self.num1 * self.num2
        return self.result


    def divide(self): 
        self.operation = '/'
        self.result = self.num1 / self.num2
        return self.result


    def pow(self):
        self.operation = '^'
        # Issue: the original implementation multiplied by the loop index (2..num2),
        # which produced a factorial-like result instead of exponentiation (e.g., 2^3 -> 12).
        # Unit tests for exponentiation caught this. Fixed by using Python's exponent operator (**).
        self.result = self.num1 ** self.num2
        return self.result


    def print_result(self):

        print("{0} {1} {2} = {3}".format(self.num1, self.operation, 
                                         self.num2, self.result
                                  ))
        input("Press enter to continue")
        return True



class CalculatorHandler:

    def print_menu(self): 
        os.system('clear')
        print("Operations available -\n"
              "1. Add\n"
              "2. Subtract\n"
              "3. Multiply\n"
              "4. Divide\n"
              "5. Power\n"
              "6. Exit")
        return True


    def call_calculator(self, number_1, number_2, option):

        # Issue: casting to int broke decimal inputs like "2.5" and truncated values.
        # Unit tests using float strings caught this. Fixed by converting to float.
        number_1 = float(number_1)
        number_2 = float(number_2)
        calc = Calculator(number_1, number_2)

        if option == 1:
            calc.sum()
        elif option == 2:
            calc.subtract()
        elif option == 3:
            calc.multiply()
        elif option == 4:
            calc.divide()
        elif option == 5:
            calc.pow()
        else:
            raise ValueError(f"Invalid option: {option}")
        calc.print_result()

        return True


In [ ]:
def run_cli():
    """Run the interactive calculator UI.

    Note: this is wrapped in a function so that running all notebook cells does not
    block unit tests waiting for input.
    """
    handler = CalculatorHandler()
    while True:

        if handler.print_menu():
            option = int(input("Select an option:"))
            if option == 6:
                break

        number_1 = input("Enter first number: ")
        number_2 = input("Enter second number: ")

        try:
            handler.call_calculator(number_1, number_2, option)
        except Exception as exc:
            print(f"something went wrong: {exc}")


# Uncomment the line below to run the CLI interactively.
# run_cli()


## Your solution is expected in the following cells 
**Create as many code/text cells as needed**

## Test Approach and Strategy

### Testing Methodology
I will create unit tests for each Calculator method using Python's `unittest` framework. Each function will have at least 3 test cases covering:
1. **Known cases**: Standard inputs with expected outputs
2. **Edge cases**: Boundary conditions (zero, negative numbers, floats)
3. **Exception cases**: Invalid inputs that should raise errors

### Assert Types Used (6+ required)
1. `assertEqual` - Verify exact return values
2. `assertNotEqual` - Confirm values are different
3. `assertTrue` - Check boolean conditions
4. `assertAlmostEqual` - Handle floating-point precision
5. `assertIsInstance` - Verify return types
6. `assertRaises` - Test exception handling (used at least twice)
7. `assertGreater` - Compare numerical values

### Functions Being Tested
- `sum()` - Addition of two numbers
- `subtract()` - Subtraction of two numbers
- `multiply()` - Multiplication of two numbers
- `divide()` - Division (including division by zero test)
- `pow()` - Exponentiation (will reveal a critical bug)
- `call_calculator()` - Handler integration (input conversion and option routing)

In [ ]:
import unittest
from unittest.mock import patch


class TestCalculator(unittest.TestCase):
    def test_sum_known_integers(self):
        """Known case: integer addition returns the expected value."""
        calc = Calculator(2, 3)
        self.assertEqual(calc.sum(), 5)

    def test_sum_with_negative(self):
        """Edge case: addition with a negative operand works correctly."""
        calc = Calculator(-5, 2)
        self.assertEqual(calc.sum(), -3)

    def test_sum_float_precision(self):
        """Edge case: floating-point addition is close to expected value."""
        calc = Calculator(0.1, 0.2)
        self.assertAlmostEqual(calc.sum(), 0.3, places=7)
        self.assertIsInstance(calc.result, float)

    def test_subtract_known_integers(self):
        """Known case: integer subtraction returns the expected value."""
        calc = Calculator(10, 4)
        self.assertEqual(calc.subtract(), 6)

    def test_subtract_to_zero(self):
        """Edge case: subtraction resulting in zero returns 0."""
        calc = Calculator(5, 5)
        self.assertEqual(calc.subtract(), 0)
        self.assertNotEqual(calc.result, 1)

    def test_subtract_negative_numbers(self):
        """Edge case: subtracting a negative number increases the result."""
        calc = Calculator(-2, -3)
        self.assertEqual(calc.subtract(), 1)
        self.assertGreater(calc.result, 0)

    def test_multiply_known_integers(self):
        """Known case: integer multiplication returns the expected value."""
        calc = Calculator(4, 5)
        self.assertEqual(calc.multiply(), 20)

    def test_multiply_by_zero(self):
        """Edge case: multiplying by zero returns zero."""
        calc = Calculator(123, 0)
        self.assertEqual(calc.multiply(), 0)
        self.assertTrue(calc.result == 0)

    def test_multiply_negative(self):
        """Edge case: multiplying with a negative number preserves sign rules."""
        calc = Calculator(-2, 3)
        self.assertEqual(calc.multiply(), -6)
        self.assertNotEqual(calc.result, 6)

    def test_divide_known_integers(self):
        """Known case: integer division returns the expected value."""
        calc = Calculator(8, 2)
        self.assertEqual(calc.divide(), 4)

    def test_divide_fractional_result(self):
        """Known case: division can produce a fractional (float) result."""
        calc = Calculator(5, 2)
        self.assertAlmostEqual(calc.divide(), 2.5)
        self.assertIsInstance(calc.result, float)

    def test_divide_by_zero_raises(self):
        """Exception case: division by zero raises ZeroDivisionError."""
        calc = Calculator(1, 0)
        with self.assertRaises(ZeroDivisionError):
            calc.divide()

    def test_pow_known_integers(self):
        """Known case: integer exponentiation returns the expected value."""
        calc = Calculator(2, 3)
        self.assertEqual(calc.pow(), 8)

    def test_pow_exponent_zero(self):
        """Edge case: any non-zero number to the power of 0 should be 1."""
        calc = Calculator(7, 0)
        self.assertEqual(calc.pow(), 1)

    def test_pow_negative_exponent(self):
        """Edge case: negative exponents should produce fractional results."""
        calc = Calculator(2, -2)
        self.assertAlmostEqual(calc.pow(), 0.25)
        self.assertTrue(calc.result < 1)


class TestCalculatorHandler(unittest.TestCase):
    def test_call_calculator_addition_with_float_strings(self):
        """Known case: option 1 adds decimal inputs passed as strings."""
        captured = []

        def fake_print_result(self):
            captured.append((self.num1, self.operation, self.num2, self.result))
            return True

        handler = CalculatorHandler()
        with patch.object(Calculator, "print_result", new=fake_print_result):
            self.assertTrue(handler.call_calculator("2.5", "1.5", 1))

        self.assertEqual(captured[0], (2.5, "+", 1.5, 4.0))

    def test_call_calculator_division_sets_operation_and_result(self):
        """Known case: option 4 performs division and sets operation to '/'."""
        captured = []

        def fake_print_result(self):
            captured.append((self.operation, self.result))
            return True

        handler = CalculatorHandler()
        with patch.object(Calculator, "print_result", new=fake_print_result):
            handler.call_calculator("10", "4", 4)

        self.assertEqual(captured[0][0], "/")
        self.assertAlmostEqual(captured[0][1], 2.5)

    def test_call_calculator_invalid_option_raises(self):
        """Exception case: unsupported menu options raise ValueError."""
        handler = CalculatorHandler()
        with self.assertRaises(ValueError):
            handler.call_calculator("1", "2", 99)


unittest.main(argv=["first-arg-is-ignored"], verbosity=2, exit=False)


## Bugs Found and Fixes

- **`Calculator.pow()`**: The original implementation multiplied by the loop index (2..n), which produced a factorial-like result instead of exponentiation (for example, 2^3 returned 12). The unit tests for `2^3`, `x^0`, and negative exponents failed, so I replaced the logic with `self.num1 ** self.num2`.
- **`CalculatorHandler.call_calculator()`**: The original implementation converted inputs to `int`, which broke decimal inputs (e.g., "2.5") and truncated values. Unit tests using float strings caught this, so I switched to `float(...)` conversion and added a `ValueError` for invalid menu options.
